# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint
from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        t_start = time.perf_counter()
        res = func(*args, **kwargs)
        t_end = time.perf_counter()
        print(f"Время выполнения функции {func.__name__}: {t_end - t_start}")
        return res
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(f"Функция была вызвана с параментарми: {args}, {kwargs}")
        # print (func.__name__, args, kwargs)
        return res
    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print(f"Функция была вызвана: {wrapper.count} раз")
        return res
    wrapper.count = 0
    return wrapper

## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы *args должны быть хешируемыми
    """
    cache = {}

    @wraps(func)
    def wrapper(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    wrapper.cache = cache
    return wrapper

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 1.6241763001307845
Функция была вызвана с параментарми: ('whole',), {}
Функция была вызвана: 1 раз
Cлово whole встречается 176 раз


In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
start = time.perf_counter()
result = fib(20)
end = time.perf_counter()
print(f'{fib.__name__} потребовалось {end - start:.6f} секунд')

fib потребовалось 0.001598 секунд


In [10]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [12]:
# измеряем время выполнения
start = time.perf_counter()
result = fib(20)
end = time.perf_counter()
print(f'{fib.__name__} потребовалось {end - start:.6f} секунд')

fib потребовалось 0.000112 секунд
